In [41]:
%matplotlib inline
import numpy as np
import h5py 
from sklearn.metrics import f1_score,jaccard_similarity_score, accuracy_score
from sklearn.metrics import precision_recall_fscore_support as pRf1
import glob
import cv2
from scipy.optimize import linear_sum_assignment
import sys
from skimage import io
############### Visualization

from ipywidgets import Button, Layout
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display,clear_output,Video
from copy import copy
import numpy.ma as ma
import matplotlib.colors as colors
import imageio
################# Load modules
sys.path.insert(0,'/media/DATA/jbonato/astro_segm/AstroSS/modules/')

from score_fun import *
N = 256
M=256

# Dataset-1 vs. consensus

In [54]:
### uery files
GT_path = '/media/DATA/jbonato/astro_segm/set1/dataset/'
res_path = '/media/DATA/jbonato/astro_segm/Results/D1/'

masks_GT = glob.glob(GT_path+'*.*')
masks_GT.sort()
masks_GT

['/media/DATA/jbonato/astro_segm/set1/dataset/SMALL_001_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set1/dataset/SMALL_002_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set1/dataset/SMALL_003_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set1/dataset/SMALL_004_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set1/dataset/SMALL_005_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set1/dataset/SMALL_006_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set1/dataset/SMALL_007_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set1/dataset/SMALL_008_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set1/dataset/SMALL_009_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set1/dataset/SMALL_010_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set1/dataset/SMALL_011_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set1/dataset/SMALL_012_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set1/dataset/SMALL_013_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set1/dataset/SMALL_014_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set1/dataset/SMALL_015_nf.hdf5',
 '/media/D

In [55]:
GT_mat = np.zeros((256,256,2))
Res_mat = np.zeros_like(GT_mat)

h_res = np.empty((24,3))
soma_res = []
proc_res=[]
fov=0
for GT_small in masks_GT:#[j:j+1]
    name = GT_small[-17:-8]+'D1.hdf5'
    # loading GT
    dset= h5py.File(GT_small,'r') 
    GT_mat[:,:,0] =  np.asarray(dset['Values'])
    GT_mat[:,:,1] =  np.asarray(dset['Values_soma'])
    GT_mat[:,:,0]-=GT_mat[:,:,1]
    GT_mat[GT_mat<0]=0
    GT_mat[GT_mat>1]=1


    # loading Res
    dset= h5py.File(res_path+name,'r') 
    Res_mat[:,:,0] =  np.asarray(dset['Values'])
    Res_mat[:,:,1] =  np.asarray(dset['Values_soma'])
    

    aa = score_cw(GT_mat,Res_mat)
    
    proc_res.append(aa[:,:3,:])
    soma_res.append(np.mean(aa[:,:3,:],axis=0))

    a,b,c = true_pos_l(GT_mat[:,:,1],Res_mat[:,:,1],threshold=0.5,N=256)
    h_res[fov,0]=a
    h_res[fov,1]=b
    h_res[fov,2]=c

    fov+=1

detection = h_res.copy()
f1_net = np.zeros_like(detection)
f1_net[:,0] = detection[:,0]/detection[:,2]
f1_net[:,1] = detection[:,0]/detection[:,1]
f1_net[:,2] = 2*(f1_net[:,0]*f1_net[:,1])/(f1_net[:,0]+f1_net[:,1]) 
mean_f1_net = np.mean(f1_net,axis=0)
std_f1_net = np.std(f1_net,axis=0)

final_proc = proc_res[0]
for r in range(1,len(proc_res)):
    final_proc = np.vstack((final_proc,proc_res[r]))
mean_val = np.mean(final_proc,axis=0)
print('Somata: \n P: {:.2f}, R:  {:.2f}, F1:  {:.2f}'.format(mean_val[0,0],mean_val[1,0],mean_val[2,0]))
print('Processes \n P: {:.2f}, R:  {:.2f}, F1:  {:.2f}'.format(mean_val[0,1],mean_val[1,1],mean_val[2,1]))
print(mean_f1_net)

Somata: 
 P: 0.79, R:  0.89, F1:  0.83
Processes 
 P: 0.62, R:  0.60, F1:  0.59
[0.79017857 0.87276786 0.80586774]


# Results Visualization

In [44]:
def draw_cont(mask):
    N,M,c = mask.shape
    sm_c = np.zeros((N,M))
    pr_c = np.zeros((N,M))
    contours, hierarchy = cv2.findContours(np.uint8(mask[:,:,1]), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    for j in contours:
        sm_c[j[:,0,1],j[:,0,0]]=1
        
    contours, hierarchy = cv2.findContours(np.uint8(mask[:,:,0]), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    for j in contours:
        pr_c[j[:,0,1],j[:,0,0]]=1

    return np.where(sm_c==1),np.where(pr_c==1)#sm_c,pr_c#

def draw_on_Im(image,c_sm,c_pr):
    image_pl = image.copy()
    image_pl = image_pl.astype(np.float64)
    maxim = image_pl.max()
    image_pl/=maxim
    image_pl[c_sm]=4
    image_pl[c_pr]=10
    return image_pl


def get_im(mask,image):
    sm_net,pr_net = draw_cont(mask)
    image_net = draw_on_Im(image,sm_net,pr_net)
    return image_net


In [45]:
num  = widgets.IntSlider(
    value=7,
    min=1,
    max=25,
    step=1,
    description='FOV',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)


mapped = widgets.Dropdown(
    options=[('inferno',plt.cm.inferno), ('viridis',plt.cm.viridis), ('Greys',plt.cm.gray)],
    value=plt.cm.gray,
    description='Cmap:',
    disabled=False,
)
projection =  widgets.Dropdown(
    options=['median', 'max', 'mean'],
    value='median',
    description='Projection:',
    disabled=False,
)


button = widgets.Button(
    description='Plot',
)
out=widgets.Output(layout=Layout(width='500px', height='500px'))
out2=widgets.Output(layout=Layout(width='500px', height='500px'))
button=widgets.Button(description='Next')
vbox=widgets.VBox(children=(out,num,mapped,projection,button))
hbox=widgets.HBox(children=(vbox,out2))
display(hbox)

def plot2(b=None):
    if num.value==20:
        with out:
            display('error no image present')
    else:
        path = '/media/DATA/jbonato/astro_segm/set1/'+str(num.value)+'/'
        if len(str(num.value))==1:
            zer='00'
        else:
            zer='0'
        res_path = '/media/DATA/jbonato/astro_segm/Results/D1/'
        name = 'SMALL_'+zer+str(num.value)+'D1.hdf5'
        Res_mat = np.zeros((N,M,2))
        dset= h5py.File(res_path+name,'r') 
        Res_mat[:,:,0] =  np.asarray(dset['Values'])
        Res_mat[:,:,1] =  np.asarray(dset['Values_soma'])

        it = glob.glob(path+'*')
        stack = io.imread(it[0])
        video = np.empty((stack.shape[0],256,256),dtype=np.uint8)
        for j in range(stack.shape[0]):
            img = stack[j,:,:]
            bl = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
            video[j,:,:]=bl
        
        imageio.mimwrite('test2.mp4',video, fps=30);
        with out2:
            clear_output(wait=True)
            display(Video('test2.mp4',width=500,height=500))
        if projection.value=='median':
            im = np.median(stack,axis=0)
        elif projection.value == 'mean':
            im = np.mean(stack,axis=0)
        else:
            im = np.amax(stack,axis=0)
        imC = get_im(Res_mat,im)
        f, ax = plt.subplots(1, 1, figsize=(8, 8))
        c = ['snow','violet']
        palette = copy(mapped.value)
        palette.set_over(c[0],1.0)
        palette.set_bad(c[1],1.0)
        zm = ma.masked_where(imC>5,imC)
        ax.imshow(zm,cmap=palette,norm=colors.Normalize(vmin=0,vmax=1.0))
        ax.axis('off')
        with out:
            clear_output(wait=True)
            display(f)
    
button.on_click(plot2)



# Dataset-2 vs. consensus

In [56]:
### uery files
GT_path = '/media/DATA/jbonato/astro_segm/set2/dataset/'
res_path = '/media/DATA/jbonato/astro_segm/Results/D2/'

masks_GT = glob.glob(GT_path+'*nf.hdf5')
masks_GT.sort()
masks_GT

['/media/DATA/jbonato/astro_segm/set2/dataset/SMALL_026_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set2/dataset/SMALL_027_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set2/dataset/SMALL_028_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set2/dataset/SMALL_029_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set2/dataset/SMALL_030_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set2/dataset/SMALL_031_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set2/dataset/SMALL_032_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set2/dataset/SMALL_033_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set2/dataset/SMALL_034_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set2/dataset/SMALL_035_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set2/dataset/SMALL_036_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set2/dataset/SMALL_037_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set2/dataset/SMALL_038_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set2/dataset/SMALL_039_nf.hdf5',
 '/media/DATA/jbonato/astro_segm/set2/dataset/SMALL_040_nf.hdf5',
 '/media/D

In [57]:
def dilation_fun(sp_filt,dilatation_size=3):
    element = cv2.getStructuringElement(cv2.MORPH_RECT, (2*dilatation_size + 1, 2*dilatation_size+1), (dilatation_size, dilatation_size))
    dil_sp_filt = cv2.dilate(sp_filt, element)
    q =dil_sp_filt
    q[q>1]=1
    return q

def F1_cw_nd_car(GT_o,mask,flag_t=True):
    GT = GT_o.copy()
    proc_l = []
    soma_l = []
    
    N,M,cl = GT.shape
    
    #plt.imshow(mask[:,:,0])
    if flag_t:
        mask_GT  = filter_GT(GT,mask[:,:,1])
    else:
        mask_GT  = filter_GT(GT,mask[:,:,0])
        
        
    GT*=mask_GT[:,:,np.newaxis]
    
    #plt.imshow(GT[:,:,1])
    gg=np.sum(GT,axis=2)
    gg[gg>1]=1
    gg = dilation_fun(gg)
    ret_m, labels_m = cv2.connectedComponents(np.uint8(gg))
    #print('qwqwq',ret_m-1)
    inc_mask = np.sum(mask,axis=2)
    q = dilation_fun(inc_mask)
    
    ret_s, labels_s = cv2.connectedComponents(np.uint8(q))
    #plt.imshow(labels_m)
    res = np.empty((4,))
    flag = True
    

    for j in range(1,ret_s):
        soma_buff = np.zeros((N,N))
        pts_s = np.where(labels_s==j)
        soma_buff[pts_s]=1            
        flag = False
        if flag_t:
            cnt_m = soma_buff*mask[:,:,1]
            proc_single_m = soma_buff*mask[:,:,0]
        else:
            cnt_m = soma_buff*mask[:,:,0]
            proc_single_m = soma_buff*mask[:,:,1]
        soma_num,_ = cv2.connectedComponents(np.uint8(cnt_m.copy()))
        
 
        for i in range(1,ret_m):
            GT_buff = np.zeros((N,N))
            pts_m = np.where(labels_m==i)
            GT_buff[pts_m]=1
            soma_single = GT[:,:,1]*GT_buff
            proc_single = GT[:,:,0]*GT_buff
            soma_single[soma_single>1]=1
            proc_single[proc_single>1]=1
            #print('SOMMA',np.sum(soma_single*cnt_m))
            
            if IoU(soma_single,cnt_m)>=0.5:                   
                res = pRf1(soma_single.flatten(),cnt_m.flatten(),average='binary')
                soma_l.append(res)
                res = pRf1(proc_single.flatten(),proc_single_m.flatten(),average='binary')
                proc_l.append(res)

    final_res = np.dstack((np.asarray(soma_l),np.asarray(proc_l))) 
    return final_res

def filter_GT(GT,mask):
    N=512
    ret_m, labels_m = cv2.connectedComponents(np.uint8(mask))
    maskGT_T = np.sum(GT,axis=2)
    maskGT_T[maskGT_T>1]=1
    qq = dilation_fun(maskGT_T)
    ret_s, labels_s = cv2.connectedComponents(np.uint8(qq))
    cnt=0
    final = np.zeros((512,512))
    for i in range(1, ret_s):
        soma_GT_buff = np.zeros((N,N))
        pts_s = np.where(labels_s==i)
        soma_GT_buff[pts_s]=1
        rem = soma_GT_buff.copy()
        soma_GT_buff*=GT[:,:,1]
        
        for j in range(1,ret_m):
            soma_buff = np.zeros((N,N))
            pts_m = np.where(labels_m==j)
            soma_buff[pts_m]=1
            inter = soma_buff*soma_GT_buff
            #print(IoU(soma_GT_buff.flatten(),soma_buff.flatten()))
            if np.sum(inter)==np.sum(soma_buff) or np.sum(inter)==np.sum(soma_buff):
                final+=rem*maskGT_T
            else:
                score = IoU(soma_GT_buff.flatten(),soma_buff.flatten())
                if score >= 0.2:
                    final+=rem*maskGT_T
            
    return final

In [63]:
GT_mat = np.zeros((512,512,2))
Res_mat = np.zeros_like(GT_mat)

h_res = np.empty((25,3))
soma_res = []
proc_res=[]
fov=0
res_path2 = '/media/DATA/jbonato/segm_project/U-net/Results/dense_up/'
for GT_small in masks_GT:#[j:j+1]
    name = GT_small[-17:-8]+'D2.hdf5'
    print(name)
    # loading GT
    dset= h5py.File(GT_small,'r') 
    q = np.asarray(dset['Values'])
    print(q.shape)
    GT_mat[:,:,0] =  np.asarray(dset['Values'])
    GT_mat[:,:,1] =  np.asarray(dset['Values_soma'])
    GT_mat[:,:,0]-=GT_mat[:,:,1]
    GT_mat[GT_mat<0]=0
    GT_mat[GT_mat>1]=1


    # loading Res
    dset= h5py.File(res_path+name,'r') 
    Res_mat[:,:,0] =  np.asarray(dset['Values'])
    Res_mat[:,:,1] =  np.asarray(dset['Values_soma'])
    
    ret , label = cv2.connectedComponents(np.uint8(GT_mat[:,:,1]))
    if ret-1 >1:
        aa = F1_cw_nd_car(GT_mat,Res_mat)
        proc_res.append(aa[:,:3,:])
        print(aa[:,:3,:])
        
    elif IoU(GT_mat[:,:,1],Res_mat[:,:,1])>0.5:
        res_s = np.empty((1,4,2))
        res_s[0,:,0] = pRf1(GT_mat[:,:,1].flatten(),Res_mat[:,:,1].flatten(),average='binary')
        res_s[0,:,1] = pRf1(GT_mat[:,:,0].flatten(),Res_mat[:,:,0].flatten(),average='binary')
        #print(res_s[:,:3,:])
        proc_res.append(res_s[:,:3,:])

        
    if name == 'SMALL_046D2.hdf5':

        proc_res[-1][:,:,1]=np.nan

    a,b,c = true_pos_l(GT_mat[:,:,1],Res_mat[:,:,1],threshold=0.5,N=512)
    h_res[fov,0]=a
    h_res[fov,1]=b
    h_res[fov,2]=c

    fov+=1

detection = h_res.copy()
f1_net = np.zeros_like(detection)
f1_net[:,0] = detection[:,0]/detection[:,2]
f1_net[:,1] = detection[:,0]/detection[:,1]
f1_net[:,2] = 2*(f1_net[:,0]*f1_net[:,1])/(f1_net[:,0]+f1_net[:,1]) 
mean_f1_net = np.mean(f1_net,axis=0)
std_f1_net = np.std(f1_net,axis=0)

final_proc = proc_res[0]
for r in range(1,len(proc_res)):
    final_proc = np.vstack((final_proc,proc_res[r]))
mean_val = np.nanmean(final_proc,axis=0)
print('Somata: \n P: {:.2f}, R:  {:.2f}, F1:  {:.2f}'.format(mean_val[0,0],mean_val[1,0],mean_val[2,0]))
print('Processes \n P: {:.2f}, R:  {:.2f}, F1:  {:.2f}'.format(mean_val[0,1],mean_val[1,1],mean_val[2,1]))
print(mean_f1_net)

SMALL_026D2.hdf5
(512, 512)
SMALL_027D2.hdf5
(512, 512)
SMALL_028D2.hdf5
(512, 512)
SMALL_029D2.hdf5
(512, 512)
SMALL_030D2.hdf5
(512, 512)
[[[0.7952522255192879 0.5628415300546448]
  [0.9420035149384886 0.7504553734061931]
  [0.8624296057924377 0.643247462919594]]

 [[0.7728628230616302 0.65125]
  [0.9974342527261065 0.5393374741200828]
  [0.870904508541025 0.5900339750849377]]]
SMALL_031D2.hdf5
(512, 512)
SMALL_032D2.hdf5
(512, 512)
SMALL_033D2.hdf5
(512, 512)
SMALL_034D2.hdf5
(512, 512)
SMALL_035D2.hdf5
(512, 512)
SMALL_036D2.hdf5
(512, 512)
SMALL_037D2.hdf5
(512, 512)
SMALL_038D2.hdf5
(512, 512)
SMALL_039D2.hdf5
(512, 512)
SMALL_040D2.hdf5
(512, 512)
SMALL_041D2.hdf5
(512, 512)
SMALL_042D2.hdf5
(512, 512)
SMALL_043D2.hdf5
(512, 512)
SMALL_044D2.hdf5
(512, 512)
SMALL_045D2.hdf5
(512, 512)
[[[0.8978675645342312 0.5429292929292929]
  [0.9858287122612446 0.1793160967472894]
  [0.9397944199706313 0.2695924764890282]]

 [[0.8135808748728383 0.7839506172839507]
  [0.9486951364175563 0.787

In [66]:

num  = widgets.IntSlider(
    value=7,
    min=26,
    max=50,
    step=1,
    description='FOV',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)


mapped = widgets.Dropdown(
    options=[('inferno',plt.cm.inferno), ('viridis',plt.cm.viridis), ('Greys',plt.cm.gray)],
    value=plt.cm.gray,
    description='Cmap:',
    disabled=False,
)
projection =  widgets.Dropdown(
    options=['median', 'max', 'mean'],
    value='median',
    description='Projection:',
    disabled=False,
)


button = widgets.Button(
    description='Plot',
)
out=widgets.Output(layout=Layout(width='500px', height='500px'))
out2=widgets.Output(layout=Layout(width='500px', height='500px'))
button=widgets.Button(description='Next')
vbox=widgets.VBox(children=(out,num,mapped,projection,button))
hbox=widgets.HBox(children=(vbox,out2))
display(hbox)

def plot2(b=None):
    if num.value==20:
        with out:
            display('error no image present')
    else:
        path = '/media/DATA/jbonato/astro_segm/set2/'+str(num.value)+'/'
        if len(str(num.value))==1:
            zer='00'
        else:
            zer='0'
        res_path = '/media/DATA/jbonato/astro_segm/Results/D2/'
        name = 'SMALL_'+zer+str(num.value)+'D2.hdf5'
        Res_mat = np.zeros((512,512,2))
        dset= h5py.File(res_path+name,'r') 
        Res_mat[:,:,0] =  np.asarray(dset['Values'])
        Res_mat[:,:,1] =  np.asarray(dset['Values_soma'])

        it = glob.glob(path+'*')
        stack = io.imread(it[0])
        video = np.empty((stack.shape[0],512,512),dtype=np.uint8)
        for j in range(stack.shape[0]):
            img = stack[j,:,:]
            bl = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
            video[j,:,:]=bl
        
        imageio.mimwrite('test2.mp4',video, fps=30);
        with out2:
            clear_output(wait=True)
            display(Video('test2.mp4',width=500,height=500))
        if projection.value=='median':
            im = np.median(stack,axis=0)
        elif projection.value == 'mean':
            im = np.mean(stack,axis=0)
        else:
            im = np.amax(stack,axis=0)
        imC = get_im(Res_mat,im)
        f, ax = plt.subplots(1, 1, figsize=(8, 8),dpi=400)
        c = ['snow','violet']
        palette = copy(mapped.value)
        palette.set_over(c[0],1.0)
        palette.set_bad(c[1],1.0)
        zm = ma.masked_where(imC>5,imC)
        ax.imshow(zm,cmap=palette,norm=colors.Normalize(vmin=0,vmax=1.0))
        ax.axis('off')
        with out:
            clear_output(wait=True)
            display(f)
    
button.on_click(plot2)